<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# 主数据加载管道总结

完整章节代码位于 [ch02.ipynb](./ch02.ipynb)。

该笔记本包含主要的要点，即不包含中间步骤的数据加载管道。

本笔记本中使用的包：

In [1]:
# NBVAL_SKIP
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.6.0
tiktoken version: 0.9.0


In [3]:
import tiktoken  # 导入OpenAI的tokenizer库，用于将文本转换为token ID
import torch  # 导入PyTorch库，用于构建和训练神经网络
from torch.utils.data import Dataset, DataLoader  # 导入PyTorch的数据处理类


class GPTDatasetV1(Dataset):
    """
    GPT数据集类，继承自PyTorch的Dataset类。
    用于将文本数据处理成GPT模型训练所需的格式。
    
    参数:
    - txt: 输入的原始文本
    - tokenizer: 用于将文本转换为token ID的分词器
    - max_length: 每个训练样本的最大长度
    - stride: 滑动窗口的步长，用于创建重叠的训练样本
    """
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []  # 存储输入序列
        self.target_ids = []  # 存储目标序列（预测目标）

        # 使用tokenizer将整个文本编码为token ID
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # 使用滑动窗口将文本分割成重叠的序列
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]  # 输入序列
            target_chunk = token_ids[i + 1: i + max_length + 1]  # 目标序列（输入序列向右移动一位）
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        """
        返回数据集中样本的数量
        """
        return len(self.input_ids)

    def __getitem__(self, idx):
        """
        返回指定索引的样本对（输入序列和目标序列）
        """
        return self.input_ids[idx], self.target_ids[idx]


def create_dataloader_v1(txt, batch_size, max_length, stride,
                         shuffle=True, drop_last=True, num_workers=0):
    """
    创建用于GPT模型训练的DataLoader。
    
    参数:
    - txt: 输入的原始文本
    - batch_size: 每个批次的样本数量
    - max_length: 每个样本的最大长度
    - stride: 滑动窗口的步长
    - shuffle: 是否在每个epoch打乱数据
    - drop_last: 如果最后一个batch不完整，是否丢弃
    - num_workers: 用于数据加载的子进程数量
    
    返回:
    - dataloader: PyTorch的DataLoader对象，用于批量加载数据
    """
    # 初始化tokenizer，使用GPT-2的编码方式
    tokenizer = tiktoken.get_encoding("gpt2")

    # 创建数据集
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # 创建数据加载器
    dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)

    return dataloader


# 读取文本文件
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

# 设置模型参数
vocab_size = 50257  # 词汇表大小，与GPT-2的词汇表大小一致
output_dim = 256  # 嵌入向量的维度
context_length = 1024  # 上下文长度，即模型能处理的最大序列长度

# 创建token嵌入层，将token ID映射到向量空间
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
# 创建位置嵌入层，为序列中的每个位置创建一个唯一的向量
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

# 设置批处理参数
batch_size = 8  # 每个批次的样本数
max_length = 4  # 每个样本的长度
# 创建数据加载器
dataloader = create_dataloader_v1(
    raw_text,
    batch_size=batch_size,
    max_length=max_length,
    stride=max_length  # 步长等于最大长度，意味着样本之间没有重叠
)

In [4]:
for batch in dataloader:
    x, y = batch

    token_embeddings = token_embedding_layer(x)
    pos_embeddings = pos_embedding_layer(torch.arange(max_length))

    input_embeddings = token_embeddings + pos_embeddings

    break

In [5]:
print(input_embeddings.shape)

torch.Size([8, 4, 256])
